# cesm Rainfall Anomalies

In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.timePeriod as tp

In [2]:
import xarray
import numpy
import cftime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.offsetbox import AnchoredText
import cartopy.io.shapereader as shpr

%matplotlib inline

In [3]:
secondsPerDay = 60*60*24
secondsToTimeP = secondsPerDay*1000 #seconds per day, convert m to mm
#secondsToTimeP = secondsPerDay*_index.nDaysOfInterest["precAnom"]*1000/7 #seconds per month

# Calculate a climatology

Based on the control run, calculate monthly anomalies

In [ ]:
preclDa = fh.loadModelData('CESM-LME', 'PRECL', _model.cesmCntl)
preccDa = fh.loadModelData('CESM-LME', 'PRECC', _model.cesmCntl)


prectDa = (preclDa.PRECL+preccDa.PRECC)
prectDa.name='prect'

In [ ]:
monMeansDa=prectDa.groupby('time.month').mean(dim='time')

monMeansDa.where(
        (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
        drop=True
    ).load()

# Carve out an area of interest, and calculate anomalies

In [ ]:

for experiment in [*_model.cesmFullForcings, _model.cesmCntl]:
    
    #load it
    preclDa = fh.loadModelData('CESM-LME', 'PRECL', experiment)
    preccDa = fh.loadModelData('CESM-LME', 'PRECC', experiment)
    prectDa = (preclDa.PRECL+preccDa.PRECC)
    prectDa.name='prect'
    
    #anom
    anomDa=prectDa.groupby('time.month')-monMeansDa
    anomDa.name='precAnom'
        
    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )
    
    # convert to mm/year and store
    domainDa.to_netcdf('results/cesmMonthlyPrec/'+experiment+'.nc')
    
     #rcp85 extensions
    if experiment in ['002','003', '008', '009']: 
        
        #extensions
        preclDa = fh.loadModelData('CESM-LME', 'PRECL', 'LME.'+experiment)
        preccDa = fh.loadModelData('CESM-LME', 'PRECC', 'LME.'+experiment)
        extPrectDa = (preclDa.PRECL+preccDa.PRECC)
        extPrectDa.name='prect'
        
        prectDa = xarray.concat([prectDa, extPrectDa], 'time')


        #anom
        anomDa=prectDa.groupby('time.month')-monMeansDa
        anomDa.name='precAnom'

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )

        #  store
        domainDa.to_netcdf('results/cesmMonthlyPrec/LME.'+experiment+'.nc')
        
    
    

# Time averaging for the anomalies

In [ ]:
monthlyAnomDa=xarray.concat([
    xarray.open_dataset('results/cesmMonthlyPrec/'+iExp+'.nc') for iExp in _model.cesmAll],
    'experiment')
    

monthlyAnomDa=monthlyAnomDa.assign({'experiment':_model.cesmAll})


warmSeasonAnomDa=secondsToTimeP*tp.averageForTimePeriod(monthlyAnomDa)
warmSeasonAnomDa = warmSeasonAnomDa.assign_attrs([*warmSeasonAnomDa.attrs, ('units','mm/month'), ('timePeriod','Warm Season')])

In [ ]:
warmSeasonAnomDa.to_netcdf('results/cesmPrecAnoms.nc')

# Do that all again for Ts

In [ ]:
tsDa = fh.loadModelData('CESM-LME', 'TS', _model.cesmCntl)

monMeansDa=tsDa.TS.groupby('time.month').mean(dim='time')

monMeansDa.where(
        (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
        drop=True
    ).load()

# Carve out an area of interest, and calculate anomalies

In [ ]:

for experiment in [*_model.cesmFullForcings, _model.cesmCntl]: #
    
    #load it
    tsDa = fh.loadModelData('CESM-LME', 'TS', experiment).TS
    
    #anom
    anomDa=tsDa.groupby('time.month')-monMeansDa
    anomDa.name='tsAnom'
        
    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )
    
    # convert to mm/year and store
    domainDa.to_netcdf('results/cesmMonthlyTs/'+experiment+'.nc')
    
     #rcp85 extensions
    if experiment in ['002','003', '008', '009']: 
        
        #extensions
        extTsDa = fh.loadModelData('CESM-LME', 'TS', 'LME.'+experiment)
        tsDa=xarray.concat([tsDa, extTsDa.TS], 'time')

        #anom
        anomDa=tsDa.groupby('time.month')-monMeansDa
        anomDa.name='tsAnom'

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )

        domainDa.to_netcdf('results/cesmMonthlyTs/LME.'+experiment+'.nc')
        
    
    

# Time averaging for the anomalies

In [ ]:
monthlyAnomDa=xarray.concat([
    xarray.open_dataset('results/cesmMonthlyTs/'+iExp+'.nc') for iExp in _model.cesmAll],
    'experiment')
    

monthlyAnomDa=monthlyAnomDa.assign({'experiment':_model.cesmAll})

warmSeasonAnomDa=tp.averageForTimePeriod(monthlyAnomDa)

warmSeasonAnomDa.to_netcdf('results/cesmTsAnoms.nc')